In [2]:
import numpy as np
import pandas as pd
import os
import pickle

In [3]:
def find_files(directory, filename):
    """
    Search for files with a specific name within all subdirectories of a given directory.

    Parameters:
        directory (str): The root directory from which to start the search.
        filename (str): The name of the file to search for.

    Returns:
        list: A list of full paths to the files that match the given filename.
    """
    matches = []  # List to store the full paths of matching files
    for root, dirs, files in os.walk(directory):
        if filename in files:
            full_path = os.path.join(root, filename)
            matches.append(full_path)
    return matches

# Usage example
root_directory = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/'  # Update this to the directory you want to search

def load_datav1(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return set(data['V1'])

In [4]:
root_directory = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/'  # Update this to the directory you want to search
target_filename = 'snps_indices_bio1.csv'
snps_indices_bio1_files = find_files(root_directory, target_filename)

In [7]:
snps_indices_bio1_files[0]

'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_6/snps_indices_bio1.csv'

In [8]:
snps_indices_bio1 = pd.read_csv(snps_indices_bio1_files[0])


In [9]:
snps_0index = snps_indices_bio1['x'] - 1 

In [10]:
## get the vcf file frothe chromosomes and positions 
import allel
## ld pruned vcf file
vcf_file = '/carnegie/nobackup/scratch/xwu/grenet/hapFIRE_updatedVCF/greneNet_final_v1.1.recode.vcf'
vcf = allel.read_vcf(vcf_file)

/home/tbellagio/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
variants = vcf['calldata/GT']
pos = vcf['variants/POS']
chrom = vcf['variants/CHROM']

In [12]:
for split_number in range(100):
    snps_indices_bio1_file = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_{split_number}/snps_indices_bio1.csv'
    output = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_{split_number}/input_clumping.txt'
    dict_rpos_index0_file = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_{split_number}/dict_rpos_index0.pkl'
    
    snps_indices_bio1 = pd.read_csv(snps_indices_bio1_file)
    snps_0index = snps_indices_bio1['x'] - 1 
    chr_selected = pd.Series(chrom).iloc[snps_0index.to_list()]
    pos_selected = pd.Series(pos).iloc[snps_0index.to_list()]
    pos_selected = chr_selected.astype(str) + '_' + pos_selected.astype(str)
    pos_selected = pos_selected.reset_index(drop=True)

    dict_rpos_index0 = dict(zip(pos_selected, snps_0index))

    ## save the pikl relating the index0  with the actual snp id 
    with open(dict_rpos_index0_file, 'wb') as f:
        pickle.dump(dict_rpos_index0, f)  
    
    p_values = [0.0001] * len(pos_selected)  # All p-values are set to be significantly low
    # Creating a list of tuples combining SNP IDs with their p-values
    snp_p_values = list(zip(pos_selected, p_values))
    # Writing the SNP ID and p-value pairs to a text file
    
    with open(output, 'w') as file:
        # Write the header
        file.write("SNP P\n")  # Assuming you want headers named 'SNP' and 'P'
        for snp_id, p_value in snp_p_values:
            file.write(f"{snp_id} {p_value}\n")

In [13]:
root_directory = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/'  # Update this to the directory you want to search
target_filename = 'dict_rpos_index0.pkl'
dict_rpos_index0_files = find_files(root_directory, target_filename)

In [14]:
root_directory = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/'  # Update this to the directory you want to search
target_filename = 'input_clumping.txt'
input_clumping_files = find_files(root_directory, target_filename)

In [5]:

root_directory = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/'  # Update this to the directory you want to search
target_filename = 'output_clumping.clumped'
output_clumping_files = find_files(root_directory, target_filename)

In [8]:
root_directory = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/'  # Update this to the directory you want to search
target_filename = 'snps_indices_bio1.csv'
snps_indices_bio1_files = find_files(root_directory, target_filename)

In [7]:
len(output_clumping_files)

99

In [14]:
snps_f

('/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_6/output_clumping.clumped/snps_indices_bio1.csv',)

In [22]:
for i in range(100):
    snps_f = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_{i}/snps_indices_bio1.csv'
    snps = pd.read_csv(snps_f)
    #print(len(snps))
    if len(snps) == 0:
        pass
    else:
        clumps_f = f'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out_sv/split_number_{i}/output_clumping.clumped'
        clumps = pd.read_csv(clumps_f, delim_whitespace=True)
        print(len(clumps))
        print(len(clumps)/ len(snps))

129
0.19398496240601504
1014
0.16118264186933715
237
0.1975
2600
0.10473735095069288
1411
0.15173674588665448
134
0.22445561139028475
196
0.2792022792022792
539
0.14603088593876998
5577
0.13514757912082587
151
0.360381861575179
93
0.35361216730038025
425
0.13639281129653402
174
0.2373806275579809
1260
0.08344370860927153
188
0.126770060687795
1246
0.31165582791395696
220
0.2033271719038817
129
0.20607028753993611
113
0.18167202572347266
85
0.2891156462585034
178
0.1345427059712774
381
0.3407871198568873
265
0.1628764597418562
57
0.18874172185430463
436
0.24303232998885171
179
0.30134680134680136
371
0.13446901051105473
244
0.14219114219114218
145
0.17386091127098321
1039
0.1392015005359057
221
0.1758154335719968
181
0.14715447154471545
10583
0.19689668644998046
702
0.14996795556505021
178
0.24054054054054055
453
0.18859283930058285
46
0.35658914728682173
3281
0.12671867758380967
331
0.15107256960292104
348
0.13953488372093023
247
0.34691011235955055
685
0.15856481481481483
531
0.176705

In [ ]:
plink 
--bfile grenenet_og 
--clump /carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_82/input_clumping.txt 
--clump-p1 0.05 
--clump-r2 0.5 
--out /carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_82/output_clumping

In [80]:
input_clumping_files[0]

'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_82/input_clumping.txt'

In [86]:
!pwd

/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/clumping


In [89]:
clumped_data = pd.read_csv('split82.clumped', delim_whitespace=True)

In [92]:
clumped_data['F'].unique()

array([1])

In [93]:
clumped_data

,CHR,F,SNP,BP,P,TOTAL,NSIG,S05,S01,S001,S0001,SP2
0,1,1,1_194098,194098,0.0001,0,0,0,0,0,0,NONE
1,1,1,1_349186,349186,0.0001,32,0,0,0,32,0,"1_349208(1),1_349211(1),1_349223(1),1_349437(1..."
2,1,1,1_863771,863771,0.0001,0,0,0,0,0,0,NONE
3,1,1,1_902148,902148,0.0001,2,0,0,0,2,0,"1_902996(1),1_903072(1)"
4,1,1,1_909659,909659,0.0001,9,0,0,0,9,0,"1_909948(1),1_911139(1),1_911144(1),1_911148(1..."
...,...,...,...,...,...,...,...,...,...,...,...,...
567,5,1,5_26723006,26723006,0.0001,2,0,0,0,2,0,"5_26725740(1),5_26728478(1)"
568,5,1,5_26882706,26882706,0.0001,2,0,0,0,2,0,"5_26891593(1),5_26902832(1)"
569,5,1,5_26901427,26901427,0.0001,1,0,0,0,1,0,5_26901516(1)
570,5,1,5_26901571,26901571,0.0001,0,0,0,0,0,0,NONE


In [70]:
pos_selected = pos_selected.to_list()

In [ ]:
genotype_counts_filt = genotype_counts.iloc[snps_0index.to_list(), :]